In [1]:
%reload_ext autoreload
%autoreload 2

In [3]:
%run "../red_tables/data/0.1-drac-make_dataset.ipynb"
import red_tables.utils.paths as path
import tensorflow as tf
import time
import numpy as np

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


ModuleNotFoundError: No module named 'red_tables'

ModuleNotFoundError: No module named 'red_tables'

In [9]:
print("GPUs available:", len(tf.config.list_physical_devices('GPU')))

GPUs available: 0


In [ ]:
def block(x,base_filter=32):
  x = layers.Conv2D(base_filter,
                    (3,3),
                    activation="relu")(x)
  x = layers.BatchNormalization()(x)
  x = layers.MaxPooling2D(2)(x)
  return x

kernel_regularizer = l1_l2(params['lambda1'],params['lambda2'])

inputs = keras.Input(shape=(100, 100, 3))
x = inputs

x = block(x,base_filter=64)
x = block(x,base_filter=128)

x = layers.GlobalAveragePooling2D()(x)
x = layers.Dropout(params['porcentaje'])(x)
x = layers.Flatten()(x)

x = layers.Dense(200, activation = 'relu')(x)
x = layers.Dropout(params['porcentaje'])(x)
x = layers.Dense(300, activation = 'relu',kernel_regularizer=kernel_regularizer)(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model = keras.Model(inputs=inputs, outputs=outputs)

model.summary()

In [ ]:
filepath = "best_model.hdf5"

# initialize the ModelCheckpoint callback
checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
earlystop = EarlyStopping(monitor='val_accuracy',
                          mode='max',
                          restore_best_weights=True,
                          patience=20,verbose=1)
mlflow.set_experiment(experiment_name="Experiment 3")
mlflow.tensorflow.autolog()

In [ ]:
mlflow.start_run()

model.compile(loss=params['loss'],
              optimizer=Adam(
                  learning_rate=params['learning_rate'],
                  beta_1=params['beta_1'],
                  beta_2=params['beta_2'],
                  epsilon=params['epsilon'],
                  amsgrad=True
              ),
              metrics=['accuracy'])

history=model.fit(train,
                  steps_per_epoch=epoch_steps,
                  batch_size=params['batch_size'],
                  epochs=params['epochs'],
                  validation_data=test,
                  validation_steps=test_steps,
                  verbose=1,
                  callbacks=[earlystop,checkpoint])
# Guardar el nuevo modelo localmente
#mlflow.keras.save_model(model, f'nombre')
#mlflow.keras.log_model(history,'model')
mlflow.log_artifact('best_model.hdf5', "models")
mlflow.end_run()

In [ ]:
mlflow.end_run()